In [1]:
import pandas as pd
from pymongo import MongoClient
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier

from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
import nltk
from nltk.corpus import stopwords

In [2]:
# Descargar las stopwords en español
nltk.download('stopwords')
spanish_stopwords = stopwords.words('spanish')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\antho\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Configuración de MongoDB
MONGO_URI = "mongodb://localhost:27017/"  # Cambia esto si usas una configuración diferente
DATABASE_NAME = "tesis"
COLLECTION_NAME = "ciberbullying"

In [4]:
# --- Análisis de Procesamiento de Lenguaje Natural (PLN) ---

# Cargar datos desde MongoDB
def fetch_data_from_mongo(db_name, collection_name):
    """
    Extrae datos de una colección MongoDB y los convierte en un DataFrame.
    """
    client = MongoClient(MONGO_URI)
    db = client[db_name]
    collection = db[collection_name]
    data = pd.DataFrame(list(collection.find()))
    client.close()
    return data

data = fetch_data_from_mongo(DATABASE_NAME, COLLECTION_NAME)

In [5]:
# Limpieza de datos
def clean_data(data, text_column):
    """
    Limpia el DataFrame eliminando o reemplazando valores nulos en la columna de texto.

    Parameters:
        data (DataFrame): Datos con texto y etiquetas.
        text_column (str): Nombre de la columna de texto.

    Returns:
        DataFrame: Datos limpios.
    """
    data[text_column] = data[text_column].fillna("")  # Reemplazar NaN con cadena vacía
    return data

# Limpieza de los datos antes del preprocesamiento
data = clean_data(data, "texto")

In [6]:
# Preprocesamiento del texto
def preprocess_text(data, text_column, label_columns):
    """
    Preprocesa los textos y divide en conjuntos de entrenamiento y prueba.

    Parameters:
        data (DataFrame): Datos con columnas de texto y etiquetas.
        text_column (str): Nombre de la columna con texto.
        label_columns (list): Nombres de las columnas de etiquetas.

    Returns:
        X_train, X_test, y_train, y_test: Conjuntos de datos divididos.
    """
    vectorizer = TfidfVectorizer(stop_words=spanish_stopwords)
    X = vectorizer.fit_transform(data[text_column])
    y = data[label_columns]
    return train_test_split(X, y, test_size=0.2, random_state=42)

text_column = "texto"  # Columna de texto
label_columns = [
    "insulto", "amenaza", "rumores_difamatorios", "comentarios_humillantes", "lenguaje_discriminatorio",
    "acoso_sexual", "manipulacion_emocional", "lenguaje_vulgar", "acoso_general", "redes_sociales",
    "suicidio_autolesion"
]  # Columnas de etiquetas

In [7]:
# Preprocesamiento
X_train, X_test, y_train, y_test = preprocess_text(data, text_column, label_columns)

In [ ]:
# --- Modelo de Machine Learning: Random Forest Multietiqueta ---
rf_base_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model = MultiOutputClassifier(rf_base_model)
rf_model.fit(X_train, y_train)  # Entrena el modelo con los datos de entrenamiento

In [ ]:
# Predicciones
rf_predictions = rf_model.predict(X_test)

# Evaluación del modelo
print("Reporte de métricas para Random Forest (multietiqueta):")
for i, column in enumerate(label_columns):
    print(f"Etiqueta: {column}")
    print(classification_report(y_test.iloc[:, i], rf_predictions[:, i]))